In [1]:
cd ..

/home/is/akiyoshi-n/my-project


In [2]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification,load_dataset_4class_Multi_classification, split_test_data_stratify, split_data_by_index, load_dataset_3class_Multi_classification, split_multilabel_data, split_same_texts_data
from src.my_project.train_v2 import ActivityPrediction, ActClassifier, MultiClassClassifier
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [4]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 5
# 早期停止のための忍耐値
PATIENCE = 2
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 4
# 閾値
THRESH = 0.5

In [5]:
from transformers import BertModel, BertConfig

# モデルの設定を読み込む
config = BertConfig.from_pretrained('/home/is/akiyoshi-n/my-project/outputs/2024-05-07/cl-tohoku/bert-base-japanese-v3/2024-05-07_2class_basic_model')

# 保存した重みを用いてモデルを読み込む
model = BertForSequenceClassification.from_pretrained('/home/is/akiyoshi-n/my-project/outputs/2024-05-07/cl-tohoku/bert-base-japanese-v3/2024-05-07_2class_basic_model', config=config)


OSError: /home/is/akiyoshi-n/my-project/outputs/2024-05-07/cl-tohoku/bert-base-japanese-v3/2024-05-07_2class_basic_model does not appear to have a file named config.json. Checkout 'https://huggingface.co//home/is/akiyoshi-n/my-project/outputs/2024-05-07/cl-tohoku/bert-base-japanese-v3/2024-05-07_2class_basic_model/main' for available files.

In [ ]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'


In [8]:
single_raw_data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")
# データの読み込み
multi_data, class_name = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")
# testデータと訓練に使用するデータに分割
multi_data, multi_test_data = split_multilabel_data(data=multi_data, test_size=0.1, SEED=SEED)

In [9]:
dataset_labels_np = np.array(multi_data['labels'])
test_data_labels_np = np.array(multi_test_data['labels'])
print(dataset_labels_np.sum(axis=0))
print(test_data_labels_np.sum(axis=0))

[110 130 322 550]
[12 15 36 61]


In [24]:
single_data = split_same_texts_data(multi_data, single_raw_data)
single_test_data = split_same_texts_data(multi_test_data, single_raw_data)

In [25]:
single_train_data, single_eval_data, a, b = split_test_data_stratify(single_data, test_size=0.3, SEED=SEED)

In [26]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
# クラス数
NUM_LABELS = 2
Classifier_model_2class = ActClassifier(model_name = MODEL_NAME, num_labels=NUM_LABELS, seed=SEED)

In [27]:
model1 = Classifier_model_2class.train_model(single_train_data, single_eval_data, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='ActClassification', run_name='test')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/755 [00:00<?, ? examples/s]

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.684000,0.697716,0.537037,0.534893
2,0.598600,0.639004,0.663580,0.662858
3,0.483200,0.651982,0.666667,0.662018
4,0.345800,0.733091,0.709877,0.704425


In [33]:
from transformers import AutoTokenizer
from src.my_project.dataset import preprocess_for_Trainer
import numpy as np
# tokenizerの定義
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# データセットの前処理
test_dataset = {'texts': multi_test_data['texts']}

prediction_label = [[0 for i in range(4)] for j in range(len(test_dataset))]

test_dataset = preprocess_for_Trainer(test_dataset, tokenizer, max_len=MAX_LEN)
pred_proba_2class = model1.predict(test_dataset)
# 2classの大きい方の確率を予測値とする
prediction_2class = np.argmax(pred_proba_2class.predictions, axis=-1)

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

In [28]:
# 活動なしのデータを含めたデータを活動ありのみのデータに変換する関数
def filter_and_remove_last_column(data):
    # 最終列が1でない行のみを保持
    filtered_data = [(text, label) for text, label in zip(data['texts'], data['labels']) if label[-1] != 1]

    # 'texts'と'labels'を更新し、最終列を削除
    updated_texts, updated_labels = zip(*[(text, label[:-1]) for text, label in filtered_data])

    # 新しい辞書を作成して返す
    return {
        'texts': list(updated_texts),
        'labels': list(updated_labels)
    }

In [30]:
# testデータと訓練に使用するデータに分割
multi_train_data, multi_eval_data = split_multilabel_data(data=multi_data, test_size=0.3, SEED=SEED)

In [32]:
# ステップ２: 細かい行動のモデル作成
# '活動あり'のデータのみで細かい活動を予測するためのインスタンス作成
multi_label_classifier = MultiClassClassifier(MODEL_NAME, SEED, 3, THRESH)
# multi用のデータから'活動なし'のデータを除外.かつ，最終列を削除
multi_train_dataset = filter_and_remove_last_column(multi_train_data)
multi_eval_dataset = filter_and_remove_last_column(multi_eval_data)
# multi用のモデル作成
model2 = multi_label_classifier.train_model(multi_train_dataset, multi_eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='a')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/369 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.642400,0.634513,0.720000,0.279070
2,0.586300,0.603801,0.680412,0.272358
3,0.516900,0.563516,0.706422,0.600337
4,0.416500,0.484544,0.684932,0.617370
5,0.290900,0.410726,0.736486,0.672487
6,0.183100,0.434604,0.754839,0.732841
7,0.116500,0.527252,0.719745,0.717225


In [38]:
true_label_3class = []
use_test_data = {'texts': []}
for i in range(len(prediction_2class)):
    if prediction_2class[i] == 1:
        true_label_3class.append(multi_test_data['labels'][i][:-1])
        use_test_data['texts'].append(multi_test_data['texts'][i])
        

In [40]:
test_dataset = preprocess_for_Trainer(use_test_data, tokenizer, max_len=MAX_LEN)

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

In [41]:
import torch

In [44]:
predictions = model2.predict(test_dataset)
pred_proba_3class = predictions.predictions
pred_proba_3class = torch.sigmoid(torch.from_numpy(pred_proba_3class))
predictions = (pred_proba_3class > THRESH).float()

In [48]:
predictions = predictions.numpy()

In [49]:
# trainerの予測値とeval_dataset['labels']のAccuracyとF1を出す
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(true_label_3class, predictions)
f1 = f1_score(true_label_3class, predictions, average='macro')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1: {f1:.4f}')

Accuracy: 0.3506
F1: 0.4320


In [39]:
true_label_3class

[[0, 1, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 1],
 [1, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 1]]

In [18]:
len(single_eval_data['labels'])

324

In [12]:
model1 = Classifier_model_2class.train_model(single_train_dataset, single_eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='ActClassification', run_name='test')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.714200,0.674166,0.590000,0.523256
2,0.646100,0.626142,0.675000,0.453782
3,0.563900,0.612530,0.690000,0.523077
4,0.405600,0.644911,0.690000,0.586667
5,0.186800,0.869219,0.735000,0.589147


In [13]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
# クラス数
NUM_LABELS = 3
Classifier_model_3class = MultiClassClassifier(model_name = MODEL_NAME, num_labels=NUM_LABELS, seed=SEED, thresh=THRESH)

In [14]:
model2 = Classifier_model_3class.train_model(multi_train_dataset, multi_eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='MultiClassification', run_name='test')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.684900,0.619077,0.620000,0.317558
2,0.616600,0.572426,0.710000,0.285714
3,0.559100,0.516087,0.700000,0.481573
4,0.434300,0.437851,0.780000,0.723348
5,0.287400,0.382963,0.800000,0.753645
6,0.164200,0.370280,0.810000,0.776689
7,0.080400,0.408484,0.850000,0.794027
8,0.061200,0.448525,0.800000,0.767880


In [15]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model_4class = ActivityPrediction(model_name = MODEL_NAME, seed = SEED, num_single_labels=2, num_multi_labels=4, thresh=THRESH)

In [16]:
preds = Classifier_model_4class.predict(model1, model2, multi_eval_dataset, MAX_LEN)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [21]:
print(len(multi_eval_dataset['labels'][0]))
print(len(preds[0]))

3
4


In [17]:
Classifier_model_4class.evaluation(multi_eval_dataset['labels'], preds)

ValueError: inconsistent shapes

In [32]:
a[:,:-1]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0

In [33]:
a = np.zeros((100,4))
a[:,:-1] = multi_eval_dataset['labels']
a

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],


In [34]:
# trainerの予測値とeval_dataset['labels']のAccuracyとF1を出す
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(a, preds)
f1 = f1_score(a, preds, average='macro')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1: {f1:.4f}')

Accuracy: 0.4100
F1: 0.4562


In [1]:
preds

NameError: name 'preds' is not defined

In [20]:
from transformers import AutoTokenizer
from src.my_project.dataset import preprocess_for_Trainer
# 使用するtokenizerを定義
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# test_datasetをTrainerで使用できる形に変換
test_dataset = preprocess_for_Trainer(single_eval_dataset, tokenizer, max_len=MAX_LEN)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
moji_only_jisho = {'texts':single_eval_dataset['texts']}
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# test_datasetをTrainerで使用できる形に変換
test_dataset = preprocess_for_Trainer(moji_only_jisho, tokenizer, max_len=MAX_LEN)
# step1: '活動あり/なし'の予測
# 活動あり/なしの予測確率
pred_proba_2class = model1.predict(test_dataset)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [24]:
len(test_dataset)

200

In [23]:
pred_proba_3class = model2.predict(test_dataset)

In [45]:
# 予測ラベルを保存するリストを作成．ただし，全ての要素を0にする
# テストサイズ×マルチラベル数の二次元リスト
prediction_label = [[0 for i in range(4)] for j in range(len(test_dataset))]

In [43]:
len(test_dataset['input_ids']

100

In [46]:
# step1: '活動あり/なし'の予測
# 活動あり/なしの予測確率
pred_proba_2class = model1.predict(test_dataset)

In [17]:
model2

In [47]:
pred_proba_2class

PredictionOutput(predictions=array([[-1.29678592e-01,  2.29858577e-01],
       [ 1.42695856e+00, -5.98368347e-01],
       [-7.22448528e-02, -2.25269631e-01],
       [ 9.55716848e-01, -8.33176613e-01],
       [ 1.07089078e+00, -4.85432327e-01],
       [ 1.14154029e+00, -6.89009547e-01],
       [-3.58893096e-01, -2.94585198e-01],
       [-1.94798887e-01,  5.86966455e-01],
       [ 1.12888288e+00, -1.85428724e-01],
       [ 7.20013678e-01, -2.93522120e-01],
       [-1.16623056e+00,  3.96503061e-01],
       [ 5.58959186e-01, -6.83495700e-01],
       [-1.25392234e+00,  2.49102190e-01],
       [-1.80965900e+00,  5.45425355e-01],
       [-2.89474845e-01,  1.43729970e-01],
       [ 7.97102153e-01, -7.76707232e-01],
       [ 1.40536451e+00, -1.13220239e+00],
       [ 1.26109087e+00, -7.32190490e-01],
       [ 1.25748813e+00, -2.17474148e-01],
       [ 6.26744390e-01, -5.52786946e-01],
       [ 3.70005995e-01, -7.06388712e-01],
       [ 7.55935013e-02,  6.65948689e-01],
       [ 1.03705382e+00, 

In [ ]:
preds

In [ ]:
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
# ラベルが前から順にどういう順番かを把握する
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final.xlsx")

In [ ]:
df = pd.read_excel(f"{DATASET_PATH}/act_classification_final.xlsx")
# テキストデータとラベルデータを取得する
texts = df['text'].values.tolist()
# ラベルデータをリストのリストとして取得
labels = df[['label1', 'label2', 'label3', 'label4', 'label5']].fillna('').values.tolist()

/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# ラベルの最初の数字を取得して新しいラベルとして使用
new_labels = []
for label_list in labels:
    new_label_list = [label.split(' ')[0] if label else '' for label in label_list]
    new_labels.append(new_label_list)

In [ ]:
new_labels

[['4.', '', '', '', ''],
 ['2', '', '', '', ''],
 ['3', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['1', '', '', '', ''],
 ['3', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['3', '', '', '', ''],
 ['2', '', '', '', ''],
 ['1', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['3', '', '', '', ''],
 ['3', '', '', '', ''],
 ['3', '', '', '', ''],
 ['2', '', '', '', ''],
 ['1', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['1', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['3', '', '', '', ''],
 ['2', '', '', '', ''],
 ['2', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['2', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['4.', '', '', '', ''],
 ['3', '', '', ''

In [ ]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(new_labels)
class_name = mlb.classes_[1:]
labels_list = [row[1:] for row in labels_list] # ０列目の空白を削除

In [ ]:
labels

array([[1, 0, 0, 0, 1],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       ...,
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1]])

In [ ]:
labels

[['4. 活動なし', '', '', '', ''],
 ['2 - A - b. その他の移動', '', '', '', ''],
 ['3 - D - a. 交際-物理的接触', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['1 - C - b. 食事-飲酒なし・不明', '', '', '', ''],
 ['3 - C - a. 趣味・娯楽-体動かす', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['3 - C - a. 趣味・娯楽-体動かす', '', '', '', ''],
 ['2 - D. 家事', '', '', '', ''],
 ['1 - C - b. 食事-飲酒なし・不明', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['3 - C - a. 趣味・娯楽-体動かす', '', '', '', ''],
 ['3 - F - a. 喫煙', '', '', '', ''],
 ['3 - B. 買い物', '', '', '', ''],
 ['2 - A - a. 通勤・通学', '', '', '', ''],
 ['1 - A. 睡眠', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['1 - C - a. 食事-飲酒あり', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['4. 活動なし', '', '', '', ''],
 ['3 - D - b. 交際-オンライン的接触', '', '', '', ''],
 ['2 - A - a. 通勤・通学', '', '', '', ''],
 ['2 - B. 仕

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

def load_multiclass_dataset(DATASET_PATH, merge_labels=None):
    """
    データセットの読み込み
    merge_labels: マージしたいラベルの辞書。例: {'C': 'A'} は 'C' を 'A' にマージする
    """
    df = pd.read_excel(DATASET_PATH)

    # テキストデータとラベルデータを取得する
    texts = df['text'].values.tolist()
    
    # ラベルデータをリストのリストとして取得
    labels = df[['label1', 'label2', 'label3', 'label4', 'label5']].fillna('').values.tolist()
    
    # ラベルのマージ処理
    if merge_labels:
        for row in labels:
            for i, label in enumerate(row):
                if label in merge_labels:
                    row[i] = merge_labels[label]

    # ラベルデータをワンホットエンコーディングに変換
    mlb = MultiLabelBinarizer()
    labels = mlb.fit_transform(labels)
    class_name = mlb.classes_[1:]
    labels_list = labels.tolist()
    labels_list = [row[1:] for row in labels_list] # ０列目の空白を削除
    
    # テキストとラベルを含む辞書型のデータセットとして返す
    dataset = {
        'texts': texts,
        'labels': labels_list
    }

    return dataset, class_name

In [ ]:
# merge_dict = {'C': 'A'}
merge_dict = {'4. 活動なし': '1 - A. 睡眠', '3 - D - a. 交際-物理的接触': '1 - A. 睡眠'}
dataset, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final.xlsx", merge_labels=merge_dict)
print(class_name)

['1 - A. 睡眠' '1 - B. 身の回りの用事' '1 - C - a. 食事-飲酒あり' '1 - C - b. 食事-飲酒なし・不明'
 '2 - A - a. 通勤・通学' '2 - A - b. その他の移動' '2 - B. 仕事' '2 - C. 学業'
 '2 - D. 家事' '2 - F. 育児' '3 - A. メディア（テレビ・ラジオ・新聞・雑誌・ネット）' '3 - B. 買い物'
 '3 - C - a. 趣味・娯楽-体動かす' '3 - C - b. 趣味・娯楽-体動かさない'
 '3 - D - b. 交際-オンライン的接触' '3 - E. 受診・療養' '3 - F - a. 喫煙'
 '3 - F - b. その他の活動(事故など)']


/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
print(len(dataset['labels'][0]))
dataset['labels'][0:5]

18


[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
class_name

array(['1 - A. 睡眠', '1 - B. 身の回りの用事', '1 - C - a. 食事-飲酒あり',
       '1 - C - b. 食事-飲酒なし・不明', '2 - A - a. 通勤・通学', '2 - A - b. その他の移動',
       '2 - B. 仕事', '2 - C. 学業', '2 - D. 家事', '2 - F. 育児',
       '3 - A. メディア（テレビ・ラジオ・新聞・雑誌・ネット）', '3 - B. 買い物',
       '3 - C - a. 趣味・娯楽-体動かす', '3 - C - b. 趣味・娯楽-体動かさない',
       '3 - D - a. 交際-物理的接触', '3 - D - b. 交際-オンライン的接触', '3 - E. 受診・療養',
       '3 - F - a. 喫煙', '3 - F - b. その他の活動(事故など)', '4. 活動なし'],
      dtype=object)

In [ ]:
four_column_list = [
    [0, 0, 0, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 0, 1, 0]
]

# 3列の2次元リスト
three_column_list = [
    [9, 9, 9],
    [8, 8, 8],
    [7, 7, 7],
    [6, 6, 6],
    [5, 5, 5],
    [4, 4, 4]
]
for i in range(len(four_column_list)):
    four_column_list[i][:-1] = three_column_list[i]
four_column_list

[[9, 9, 9, 1],
 [8, 8, 8, 0],
 [7, 7, 7, 0],
 [6, 6, 6, 1],
 [5, 5, 5, 0],
 [4, 4, 4, 0]]

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

def load_multiclass_dataset(DATASET_PATH):
    """データセットの読み込み"""
    df = pd.read_excel(DATASET_PATH)

    # テキストデータとラベルデータを取得する
    texts = df['text'].values.tolist()
    
    # ラベルデータをリストのリストとして取得
    labels = df[['label1', 'label2', 'label3', 'label4', 'label5']].fillna('').values.tolist()
    
    # ラベルの最初の数字を取得して新しいラベルとして使用
    new_labels = []
    for label_list in labels:
        new_label_list = [label.split(' ')[0] if label else '' for label in label_list]
        new_labels.append(new_label_list)
    
    # ラベルデータをワンホットエンコーディングに変換
    mlb = MultiLabelBinarizer()
    labels = mlb.fit_transform(new_labels)
    class_name = mlb.classes_[1:]
    labels_list = labels.tolist()
    labels_list = [row[1:] for row in labels_list] # ０列目の空白を削除
    
    # テキストとラベルを含む辞書型のデータセットとして返す
    dataset = {
        'texts': texts,
        'labels': labels_list
    }

    return dataset, class_name

  

In [ ]:
# データの読み込み
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/act_classification_final.xlsx")

/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
class_name

array(['1', '2', '3', '4.'], dtype=object)

In [ ]:
import numpy as np

In [ ]:
import torch

In [ ]:
a = np.array([[0,2,0],[0, 4, 3],[0,2,0]])
a = torch.from_numpy(a)
print(torch.sigmoid(a))
# numpy配列に変形
(a > 0).float().numpy()

tensor([[0.5000, 0.8808, 0.5000],
        [0.5000, 0.9820, 0.9526],
        [0.5000, 0.8808, 0.5000]])


array([[0., 1., 0.],
       [0., 1., 1.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels") #keeps the labels
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight = class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

In [ ]:
import torch
import numpy as np

In [ ]:
labels = np.array([[0,1,0,0],[1,1,0,0],[0,0,0,1],[1,1,1,0]])
labels = torch.from_numpy(labels)
labels

tensor([[0, 1, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 0, 1],
        [1, 1, 1, 0]])

In [ ]:
# labelsの各クラスの個数をカウント
label_counts = labels.sum(axis=0)
label_counts

tensor([2, 3, 1, 1])

In [ ]:
len(label_counts)

4

In [ ]:
total_counts = torch.sum(label_counts)
total_counts

tensor(7)

In [ ]:
cd ..

/home/is/akiyoshi-n/my-project


In [ ]:
from src.my_project.dataset import preprocess_for_Trainer
from transformers import AutoTokenizer
from src.my_project.dataset import load_text_dataset
from pathlib import Path

In [ ]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 最大トークン数
MAX_LEN = 128

In [ ]:
model_name = MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
test_dataset = load_text_dataset(f"{DATASET_PATH}/add_data_sub.txt.xlsx")

In [ ]:
# tokenizerの定義
tokenizer = AutoTokenizer.from_pretrained(model_name)
# データセットの前処理
test_dataset = preprocess_for_Trainer(test_dataset, tokenizer, max_len=MAX_LEN)

Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, 'max_len': 128} of the transform datasets.arrow_dataset.Dataset._map_single cou

Map:   0%|          | 0/6887 [00:00<?, ? examples/s]

In [ ]:
test_dataset

Dataset({
    features: ['texts', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6887
})

In [ ]:
import torch

In [ ]:
# ここでは、10個のサンプルがあり、それぞれがクラス0、1、または2に属しているとします。
labels = torch.tensor([0, 1, 1, 0, 1, 0, 1])

In [ ]:
# クラス毎の個数を計算
label_counts = torch.bincount(labels, minlength=3)
label_counts

tensor([3, 4, 0])

In [ ]:
# 全体の個数を計算
total_counts = torch.sum(label_counts).float()
total_counts

tensor(7.)

### Cross Entropy Loss

### Binary Cross Entropy loss

In [ ]:
import torch.nn as nn

In [ ]:
loss = nn.BCELoss()
input1 = torch.tensor([0.5, 0.45, 0.05])
target1 = torch.tensor([1.0, 0, 0])
input2 = torch.tensor([0.5, 0.25, 0.25])
target2 = torch.tensor([1.0, 0, 0])
print(loss(input1, target1))
print(loss(input2, target2))

tensor(0.4474)
tensor(0.4228)


In [ ]:
# txtファイルの読み込み
import pandas as pd
from pathlib import Path

DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
column_names = ['Num_ID','Name','text','time']
df = pd.read_csv(f"{DATASET_PATH}/urso_users.txt", sep='\t', names=column_names)

In [ ]:
len(df)

510269

In [ ]:
len(df.groupby('Name'))

85

### スクレイピング

In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from tqdm import tqdm

# データを格納するリスト
all_data = []

# 初期URL
base_url = 'https://search.ameba.jp/general/entry/%E3%83%80%E3%82%A4%E3%82%A8%E3%83%83%E3%83%88.html'

# 初期ページURL
current_url = base_url

# ブログの取得数
blog_count = 0

# データを取得する関数
def get_page_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    articles = soup.find_all('li', class_='PcEntryListItem')
    
    page_data = []
    for article in articles:
        title_tag = article.find('div', class_='PcEntryListItem_EntryTitle')
        title = title_tag.get_text(strip=True) if title_tag else ''
        
        link_tag = article.find('a', class_='PcEntryListItem_Link')
        link = link_tag['href'] if link_tag else ''
        
        date_tag = article.find('span', class_='PcEntryListItem_EntryDate')
        date = date_tag.get_text(strip=True) if date_tag else ''
        
        blog_title_tag = article.find('span', class_='PcEntryListItem_BlogTitle')
        blog_title = blog_title_tag.get_text(strip=True) if blog_title_tag else ''
        
        thumbnail_tag = article.find('img', class_='LazyImage_Loading')
        thumbnail = thumbnail_tag['data-src'] if thumbnail_tag else ''
        
        # テキスト全文を取得
        full_text = get_full_text(link)
        
        page_data.append({
            'title': title,
            'link': link,
            'date': date,
            'blog_title': blog_title,
            'thumbnail': thumbnail,
            'full_text': full_text  # テキスト全文を追加
        })
    
    return page_data, soup

# 「次の10件」リンクを取得する関数
def get_next_page_url(soup):
    next_link_tag = soup.find('a', class_='PcResultPagination_MoreLink')
    if next_link_tag and 'href' in next_link_tag.attrs:
        return next_link_tag['href']
    else:
        return None

# テキスト全文を取得する関数
def get_full_text(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    text_tags = soup.find_all('p')
    full_text = ' '.join(tag.get_text(strip=True) for tag in text_tags)
    
    return full_text

number = 20
# 時間を計る
import time
start = time.time()
# データ収集ループ
with tqdm(total=number) as pbar:
    while current_url and blog_count < number:
        pbar.set_description(f"Fetching data from {current_url}")
        page_data, soup = get_page_data(current_url)
        all_data.extend(page_data)
        next_page_url = get_next_page_url(soup)
        
        # ページ番号の取得
        current_page_number = int(parse_qs(urlparse(current_url).query).get('p', [1])[0])
        
        # 次のページのURLを生成
        next_page_number = current_page_number + 1
        next_url = base_url + f'?p={next_page_number}'
        
        current_url = next_url
        blog_count += len(page_data)

        # ブログの取得数がnumber件を超えた場合、必要なエントリー数を削除する
        if blog_count > number:
            excess_entries = blog_count - number
            all_data = all_data[:-excess_entries]

        pbar.update(len(page_data))
# 時間を計る
end = time.time()
# 時間を算出
print(f"処理時間: {end - start}")
# 結果を表示
# for entry in all_data:
#     print(f"Title: {entry['title']}")
#     print(f"Link: {entry['link']}")
#     print(f"Date: {entry['date']}")
#     print(f"Blog Title: {entry['blog_title']}")
#     print(f"Thumbnail: {entry['thumbnail']}")
#     print(f"Full Text: {entry['full_text'][:500]}...")  # 最初の500文字のみ表示
#     print('-' * 50)

  0%|          | 0/20 [00:00<?, ?it/s]

Fetching data from https://search.ameba.jp/general/entry/%E3%83%80%E3%82%A4%E3%82%A8%E3%83%83%E3%83%88.html?p=2: 100%|██████████| 20/20 [01:50<00:00,  5.52s/it]

処理時間: 110.31177878379822


In [10]:
len(all_data)

20

In [11]:
all_data

[{'title': '【ごねるお爺】電話嫌いな夫にヘリ子ヤキモキ!!',
  'link': 'https://ameblo.jp/chunta-2011/entry-12854881769.html',
  'date': '2024年06月05日',
  'blog_title': 'kosodatefulな毎日 ~オギャ子の暴走~',
  'thumbnail': 'https://stat.profile.ameba.jp/profile_images/20200829/15/82/ob/j/o04200420p_1598681652885_ytnj9.jpg?cpe=176',
  'full_text': 'kosodatefulな毎日 ～オギャ子の暴走～ 笑いありドジありたまにドン引きあり！？三兄弟の子育て&紅一点で頑張る私のネタに尽きない日々を綴ったエンタメブログです！Blog of the Year2017、2018と二年連続受賞して、2019年殿堂入りしました~！やっぴー！  チャオーーーーー！！！  梅ひじきファンの皆様！  ワクエちゃんが喜んでおります(笑)  数々のおすすめ梅ひじき教えてくれてありがと～～、皆さんがおすすめしてた「えとや」さんの梅ひじきはワクエちゃんも大好きだって！  お客様が買いやすいように・・・とドドンと２・５倍の２５０ｇサイズが今１０００円ぽっきりなので是非この機会に食べてみてください！     ＧＷ中の話やからちょっと時間が経ってしまってるんやけど スシオちゃんが花園（東大阪にあるラグビーの聖地）で試合してた日のこと。   ちょ～～ど今からスシオが出るぞ！！！っていうタイミングで  四角に電話がかかってきた・・・   滅多にならない四角の電話がまさかピンポイントのこんなタイミングでかかってくるなんて。   しかもちょっとややこしい話やってさ。 （ややこしいって言ったらあれやけど）    （ちょっとニッチなものなのでふわっと介護用品とさせてください）   他に頼める人はいない。  何故なら運転できる人が四角しかいないから。   しかもＧＷ中なので院内の売店も閉まってるため「急いで買ってきてほしい・・・」と四角に頼んできたわけであ～～る。  お・・おおおお・・・  なんていうタイミングだよおお～！   今目の前でスシオ

In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# データを格納するリスト
all_data = []

# 初期URL
base_url = 'https://search.ameba.jp/general/entry/%E3%83%80%E3%82%A4%E3%82%A8%E3%83%83%E3%83%88.html'

# 初期ページURL
current_url = base_url

# ブログの取得数
blog_count = 0

# データを取得する関数
def get_page_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    articles = soup.find_all('li', class_='PcEntryListItem')
    
    page_data = []
    for article in articles:
        title_tag = article.find('div', class_='PcEntryListItem_EntryTitle')
        title = title_tag.get_text(strip=True) if title_tag else ''
        
        link_tag = article.find('a', class_='PcEntryListItem_Link')
        link = link_tag['href'] if link_tag else ''
        
        date_tag = article.find('span', class_='PcEntryListItem_EntryDate')
        date = date_tag.get_text(strip=True) if date_tag else ''
        
        blog_title_tag = article.find('span', class_='PcEntryListItem_BlogTitle')
        blog_title = blog_title_tag.get_text(strip=True) if blog_title_tag else ''
        
        thumbnail_tag = article.find('img', class_='LazyImage_Loading')
        thumbnail = thumbnail_tag['data-src'] if thumbnail_tag else ''
        
        page_data.append({
            'title': title,
            'link': link,
            'date': date,
            'blog_title': blog_title,
            'thumbnail': thumbnail,
            'full_text': ''  # テキスト全文は後で取得
        })
    
    return page_data, soup

# 「次の10件」リンクを取得する関数
def get_next_page_url(soup):
    next_link_tag = soup.find('a', class_='PcResultPagination_MoreLink')
    if next_link_tag and 'href' in next_link_tag.attrs:
        return next_link_tag['href']
    else:
        return None

# テキスト全文を取得する関数
def get_full_text(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    text_tags = soup.find_all('p')
    full_text = ' '.join(tag.get_text(strip=True) for tag in text_tags)
    
    return full_text

number = 20
# 時間を計る
start = time.time()

# 並行処理のためのスレッドプールを使用
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(get_page_data, current_url): current_url}
    with tqdm(total=number) as pbar:
        while current_url and blog_count < number:
            for future in as_completed(future_to_url):
                url = future_to_url[future]
                try:
                    page_data, soup = future.result()
                    all_data.extend(page_data)
                    next_page_url = get_next_page_url(soup)
                    
                    # ページ番号の取得
                    current_page_number = int(parse_qs(urlparse(current_url).query).get('p', [1])[0])
                    
                    # 次のページのURLを生成
                    next_page_number = current_page_number + 1
                    next_url = base_url + f'?p={next_page_number}'
                    
                    future_to_url[executor.submit(get_page_data, next_url)] = next_url
                    current_url = next_url
                    blog_count += len(page_data)

                    # ブログの取得数がnumber件を超えた場合、必要なエントリー数を削除する
                    if blog_count > number:
                        excess_entries = blog_count - number
                        all_data = all_data[:-excess_entries]

                    pbar.update(len(page_data))
                except Exception as exc:
                    print(f'Exception {exc} occurred while fetching {url}')
                    current_url = None
                    break

# テキスト全文を並行して取得
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_link = {executor.submit(get_full_text, entry['link']): entry for entry in all_data}
    for future in tqdm(as_completed(future_to_link), total=len(all_data), desc='Fetching full texts'):
        entry = future_to_link[future]
        try:
            entry['full_text'] = future.result()
        except Exception as exc:
            print(f'Exception {exc} occurred while fetching full text for {entry["link"]}')

# 時間を計る
end = time.time()
# 時間を算出
print(f"処理時間: {end - start}")
# 結果を表示
# for entry in all_data:
#     print(f"Title: {entry['title']}")
#     print(f"Link: {entry['link']}")
#     print(f"Date: {entry['date']}")
#     print(f"Blog Title: {entry['blog_title']}")
#     print(f"Thumbnail: {entry['thumbnail']}")
#     print(f"Full Text: {entry['full_text'][:500]}...")  # 最初の500文字のみ表示
#     print('-' * 50)


30it [00:21,  1.37it/s]                        
Fetching full texts: 100%|██████████| 20/20 [00:21<00:00,  1.09s/it]

処理時間: 52.03134846687317


In [14]:
all_data

[{'title': '【ごねるお爺】電話嫌いな夫にヘリ子ヤキモキ!!',
  'link': 'https://ameblo.jp/chunta-2011/entry-12854881769.html',
  'date': '2024年06月05日',
  'blog_title': 'kosodatefulな毎日 ~オギャ子の暴走~',
  'thumbnail': 'https://stat.profile.ameba.jp/profile_images/20200829/15/82/ob/j/o04200420p_1598681652885_ytnj9.jpg?cpe=176',
  'full_text': 'kosodatefulな毎日 ～オギャ子の暴走～ 笑いありドジありたまにドン引きあり！？三兄弟の子育て&紅一点で頑張る私のネタに尽きない日々を綴ったエンタメブログです！Blog of the Year2017、2018と二年連続受賞して、2019年殿堂入りしました~！やっぴー！  チャオーーーーー！！！  梅ひじきファンの皆様！  ワクエちゃんが喜んでおります(笑)  数々のおすすめ梅ひじき教えてくれてありがと～～、皆さんがおすすめしてた「えとや」さんの梅ひじきはワクエちゃんも大好きだって！  お客様が買いやすいように・・・とドドンと２・５倍の２５０ｇサイズが今１０００円ぽっきりなので是非この機会に食べてみてください！     ＧＷ中の話やからちょっと時間が経ってしまってるんやけど スシオちゃんが花園（東大阪にあるラグビーの聖地）で試合してた日のこと。   ちょ～～ど今からスシオが出るぞ！！！っていうタイミングで  四角に電話がかかってきた・・・   滅多にならない四角の電話がまさかピンポイントのこんなタイミングでかかってくるなんて。   しかもちょっとややこしい話やってさ。 （ややこしいって言ったらあれやけど）    （ちょっとニッチなものなのでふわっと介護用品とさせてください）   他に頼める人はいない。  何故なら運転できる人が四角しかいないから。   しかもＧＷ中なので院内の売店も閉まってるため「急いで買ってきてほしい・・・」と四角に頼んできたわけであ～～る。  お・・おおおお・・・  なんていうタイミングだよおお～！   今目の前でスシオ